<a href="https://githubtocolab.com/gee-community/geemap/blob/master/examples/notebooks/141_image_array_viz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/></a>

**Visualizing in-memory raster datasets and image arrays**

In [ ]:
# !pip install -U geemap

In [ ]:
import ee
import geemap

Use a Landsat image covering San Francisco Bay Area.

In [ ]:
m = geemap.Map()
image = ee.Image("LANDSAT/LC08/C02/T1_TOA/LC08_044034_20140318").select(
    ["B5", "B4", "B3"]
)
vis_params = {
    "min": 0.0,
    "max": 0.4,
}
m.add_layer(image, vis_params, "False Color (543)")
m.centerObject(image)
m

Retrieve the projection information from the image.

In [ ]:
projection = image.select(0).projection().getInfo()
crs = projection["crs"]
crs

Read the Earth Engine image into an Xarray DataArray.

In [ ]:
ds = geemap.ee_to_xarray(
    image, crs="EPSG:32610", scale=300, geometry=image.geometry(), ee_mask_value=0
)
ds

Save the DataArray as a Cloud Optimized GeoTIFF.

In [ ]:
geemap.xee_to_image(ds, filenames=["landsat.tif"])

Calculate the NDVI and save it to the DataArray.

In [ ]:
ndvi = (ds["B5"] - ds["B4"]) / (ds["B5"] + ds["B4"])

Create an in-memory raster dataset from the DataArray.

In [ ]:
ndvi_image = geemap.array_to_image(ndvi, source="landsat.tif")

Visualize the in-memory raster dataset.

In [ ]:
m.add_raster("landsat.tif", indexes=[1, 2, 3], nodata=-1, layer_name="Landsat 7")
m.add_raster(ndvi_image, colormap="Greens", layer_name="NDVI")
m

Save the in-memory raster dataset to a GeoTIFF file.

In [ ]:
geemap.array_to_image(ndvi, output="ndvi.tif", source="landsat.tif")